In [1]:
import math
from tensorboardX import SummaryWriter
import roboschool
import quadruppedEnv
sess = None

In [2]:
import tensorflow as tf
import collections
gpu_options = tf.GPUOptions(allow_growth=True,per_process_gpu_memory_fraction=0.8)

config=tf.ConfigProto(gpu_options=gpu_options)
sess = tf.InteractiveSession(config=config)

In [3]:
import gym
import wrappers

env_name = 'MountainCarContinuous-v0' #'QuadruppedWalk-v1' #$'BipedalWalker-v2'#'RoboschoolAnt-v1'
#'BipedalWalker-v2'#'RoboschoolAnt-v1' #'LunarLanderContinuous-v2'#'BipedalWalker-v2'#'LunarLander-v2' 
#'MountainCarContinuous-v0'#'CarRacing-v0'#'CartPole-v1' #'RoboschoolAnt-v1' #'CarRacing-v0' #'LunarLander-v2' #'Acrobot-v1' #
env = gym.make(env_name)
print(env.action_space.low)
print(env.action_space.high)

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
[-1.]
[1.]


/home/trrrrr/anaconda3/envs/rl/lib/python3.6/site-packages/gym/envs/registration.py:14: PkgResourcesDeprecationWarning: Parameters to load are deprecated.  Call .resolve and .require separately.
  result = entry_point.load(False)


In [4]:
import ray
ray.init(redis_max_memory=1024*1024*100, object_store_memory=1024*1024*100)

2019-05-22 09:29:13,953	INFO node.py:469 -- Process STDOUT and STDERR is being redirected to /tmp/ray/session_2019-05-22_09-29-13_21782/logs.
2019-05-22 09:29:14,063	INFO services.py:407 -- Waiting for redis server at 127.0.0.1:38592 to respond...
2019-05-22 09:29:14,177	INFO services.py:407 -- Waiting for redis server at 127.0.0.1:18775 to respond...
2019-05-22 09:29:14,185	INFO services.py:804 -- Starting Redis shard with 0.1 GB max memory.
2019-05-22 09:29:14,238	INFO node.py:483 -- Process STDOUT and STDERR is being redirected to /tmp/ray/session_2019-05-22_09-29-13_21782/logs.
2019-05-22 09:29:14,240	INFO services.py:1427 -- Starting the Plasma object store with 0.1 GB memory using /dev/shm.


{'node_ip_address': '192.168.3.104',
 'redis_address': '192.168.3.104:38592',
 'object_store_address': '/tmp/ray/session_2019-05-22_09-29-13_21782/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2019-05-22_09-29-13_21782/sockets/raylet',
 'webui_url': None}

In [ ]:
from a2c_v2 import A2CAgent
import tr_helpers
import networks
import a2c_games_configurations

a2c_config = a2c_games_configurations.car_config
#a2c_config = a2c_games_configurations.roboschoolant_config
#self, sess, env_name, observation_shape, actions_num, config = default_config
observation_shape = env.observation_space.shape
action_space = env.action_space

agent = A2CAgent(sess,'ppo_continuous', env.observation_space, False, action_space, a2c_config)

agent.train()



For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Use keras.layers.dense instead.
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
(pid=21858) 
(pid=21858) WARNING: The TensorFlow contrib module will not be included in TensorFlow 2.0.
(pid=21858) For more information, please see:
(pid=21858)   * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
(pid=21858)   * https://github.com/tensorflow/addons
(pid=21858) If you depend on functionality not listed there, please file an issue.
(pid=21858) 
(pid=21858) /home/trrrrr/anaconda3/envs/rl/lib/python3.6/site-packages/gym/envs/registration.py:14: PkgResourcesDeprecationWarning: Parameters to load are deprecated.  Call .resolve and 

Frames per seconds:  2421.106855054078
Frames per seconds:  2724.0637480167934
Frames per seconds:  2699.8508602343513
Frames per seconds:  2799.1576367890975
saving next best rewards:  -2.5472121117401905
Frames per seconds:  2785.758791665491
Frames per seconds:  2676.831826058829
Frames per seconds:  2387.3005184011135
Frames per seconds:  2696.4168120828017
Frames per seconds:  2911.551079468324
Frames per seconds:  2800.2763760170637
Frames per seconds:  2771.0655302308132
Frames per seconds:  2885.250979111833
Frames per seconds:  2728.905650307298
Frames per seconds:  2494.1622199200006
Frames per seconds:  2503.0558762670908
Frames per seconds:  2676.514046664552
Frames per seconds:  2494.57290989354
Frames per seconds:  2382.6467084969045
Frames per seconds:  2335.6144743943737
Frames per seconds:  2716.12301380016
Frames per seconds:  2688.1372731690876
Frames per seconds:  2723.094838919313
Frames per seconds:  2887.1575270685685
Frames per seconds:  2842.979489021018
Frames

In [ ]:
ray.shutdown()

In [ ]:
import numpy as np
#agent.restore('nn/ppo_continuousQuadruppedWalk-v1')

def evaluate(env,t_max=5000):
    rewards = []
    env._max_episode_steps = 1000
    env.batch_mode = False
    print('reset')
    #env = env.old_env
    s = env.reset()
    reward = 0
    for it in range(1000):
        action = agent.get_action([s], False)
        
        #ad = agent.get_action_distribution([s])
        #print(ad)
        s, r, done, _ = env.step(np.squeeze(action, axis = 0))
        #print(action)
        env.render()
        reward += r
        
            
        if done:
            break       
        
    return reward

import gym.wrappers

#env_monitor = wrappers.make_atari_deepmind(env_name, noop_max=30, skip=4)
#env_monitor = wrappers.ReallyDoneWrapper(env_monitor)
#env_monitor = gym.wrappers.Monitor(env,directory='video_ppo',force=True)

sessions = [print('reward:', evaluate(env)) for _ in range(1)]
#env_monitor.close()




In [ ]:
obs = env.reset()
obs, r, done, _ = env.step(2)

done = False
for _ in range(130):
    obs, _, done, _ = env.step(1)
    #env.render()
    obs = np.array(obs)
    if done:
        print('done')
        break
plt.title("what your network gonna see")
print (obs.shape)
plt.imshow(obs[:,:,0],interpolation='none',cmap='gray');

In [ ]:
import gym
gym.envs.registry.all()